In [1]:
!nvidia-smi

Thu Dec 29 06:44:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    34W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# mount driver
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CardiacSeg/UNet3D/CHGH

# install dependents
!pip install -q "monai-weekly[nibabel, tqdm, einops]"
!python -c "import matplotlib" || pip install -q matplotlib
!pip install -q timm
!pip install -U -q openmim
!mim install -U -q mmcv-full
!pip install ml-collections
!pip install ray
%matplotlib inline

# sync python module
%load_ext autoreload
%autoreload 2

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1jz_DGnICBmKWCr_JL904PDQdIEK0_EQG/CardiacSeg/UNet3D/CHGH
     |████████████████████████████████| 1.2 MB 4.9 MB/s 
     |████████████████████████████████| 41 kB 589 kB/s 
     |████████████████████████████████| 549 kB 5.0 MB/s 
     |████████████████████████████████| 182 kB 71.9 MB/s 
     |████████████████████████████████| 50 kB 3.3 MB/s 
     |████████████████████████████████| 237 kB 9.2 MB/s 
     |████████████████████████████████| 51 kB 7.2 MB/s 
     |████████████████████████████████| 46.0 MB 1.2 MB/s 
     |████████████████████████████████| 190 kB 4.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 77 kB 3.6 MB/s 
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94524 sha256=bba6e05b7f92ad4bcb79fe4f541a58ad4f70197c5a32cc800e73f3e9c1dec68f
  Stored in directory: /root/.cache/pip/wheel

In [5]:
import os

exp_name = 'exp_2_2_t4'
tune_mode = 'optim_adamw'
root_dir = '/content/drive/MyDrive/CardiacSeg/UNet3D/CHGH/tune_results'

model_name = 'unet3d'
root_data_dir = '/content/drive/MyDrive/CardiacSeg/dataset/CHGH'

data_dir = os.path.join(root_data_dir, 'dataset_2')
model_dir = os.path.join('./', 'models')
log_dir = os.path.join('./', 'logs')
start_epoch = 0
val_every = 20
max_epoch = 2000
pin_memory = True
test_mode = False
best_checkpoint = os.path.join(model_dir, 'best_model.pth')
final_checkpoint = os.path.join(model_dir, 'final_model.pth')
data_dicts_json = os.path.join(root_data_dir, 'data_dicts', 'exp_2_2.json')

In [9]:
# training
!PYTHONPATH=/content/drive/MyDrive/CardiacSeg python /content/drive/MyDrive/CardiacSeg/expers/chgh/tune.py \
--exp_name={exp_name} \
--local_dir={root_dir} \
--model_name={model_name}\
--data_dir={data_dir} \
--model_dir={model_dir} \
--log_dir={log_dir} \
--start_epoch=0 \
--val_every=20 \
--max_epoch=8000 \
--optim_lr=1e-4 \
--pin_memory \
--num_fold=3 \
--fold=2 \
--split_train_ratio=0.75 \
--data_dicts_json={data_dicts_json} \
--max_early_stop_count=10 \
--tune_mode={tune_mode} \
--checkpoint={final_checkpoint} \
#--resume_tuner

2022-12-29 07:18:46,343 - Generating grammar tables from /usr/lib/python3.8/lib2to3/Grammar.txt
2022-12-29 07:18:46,360 - Generating grammar tables from /usr/lib/python3.8/lib2to3/PatternGrammar.txt
/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
2022-12-29 07:18:48,418 - Initializing Ray automatically.For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run`.
2022-12-29 07:18:49,884	INFO worker.py:1538 -- Started a local Ray instance.
2022-12-29 07:18:51,704	INFO tensorboardx.py:170 -- pip install "ray[tune]" to see TensorBoard files.
2022-12-29 07:18:51,704	WARNING callback.py:108 -- The 